In [1]:
import scipy.io, math, os
import os
import pandas as pd, numpy as np
import torch
import matplotlib.pyplot as plt
import librosa, librosa.display

C:\Users\nchau\anaconda3\envs\env_pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# later among participants Cepstral Mean and Variance Normalization (CMVN) and Vocal Tract Length Normalization (VTLN) to normalize the feature and to reduce the differences among speakers
directory = 'data/Data/F1/mat'
counter = 1
UL_df, LL_df, JW_df, TD_df, TB_df, TT_df, audio_df = [], [], [], [], [], [], []

# Go through all the ema files and append a list so each file is found in one place
for filename in sorted(os.listdir(directory)):
    if filename.endswith('.mat'):
        f = os.path.join(directory, filename)
        mat = scipy.io.loadmat(f)
        # takes the data that is stored at the key that precedes the data for each .mat file
        data = mat['usctimit_ema_f1_{:03}_{:03}'.format(counter, counter + 4)]
        counter += 5
        
        for i in range(0,7):
            append_list(i)

# for each ema markers and audio you will have one dataframe which has the file code and appended file data 
merged_ul = preprocess(UL_df, 'ul')      
merged_ll = preprocess(LL_df, 'll')
merged_jw = preprocess(JW_df, 'jw')  
merged_td = preprocess(TD_df, 'td') 
merged_tb = preprocess(TB_df, 'tb') 
merged_tt = preprocess(TT_df, 'tt') 
merged_audio = preprocess_audio(audio_df) 

In [ ]:
a = test[0][0].values
b = ema_frames[3100]['Audio'][0][0].values
print(b)
def logmelfilterbank(audio,
                     sampling_rate,
                     fft_size=1024,
                     hop_size=256,
                     win_length=None,
                     window="hann",
                     num_mels=80,
                     fmin=None,
                     fmax=None,
                     eps=1e-10,
                     ):
    ''' essentially the same as librosa.feature.melspectrogram + log10 '''

    # get amplitude spectrogram
    x_stft = librosa.stft(audio, n_fft=fft_size, hop_length=hop_size,
                          win_length=win_length, window=window, pad_mode="reflect")
    spc = np.abs(x_stft).T  # (#frames, #bins)

    # get mel basis
    fmin = 0 if fmin is None else fmin
    fmax = sampling_rate / 2 if fmax is None else fmax
    mel_basis = librosa.filters.mel(sampling_rate, fft_size, num_mels, fmin, fmax)

    return np.log10(np.maximum(eps, np.dot(spc, mel_basis.T)))

tt = logmelfilterbank(b,22050)
tt

In [5]:
# initalise an ema list, in this append each word information in the form of dictionary
ema_frames = list()

with open('timestamps.txt', 'r') as file:
    timestamps = file.read().splitlines()
    for idx, line in enumerate(timestamps):
        print(idx)
        split_line = line.split(',')
        sent_number = int(split_line[-1])
        file_code = int(split_line[0])
        
        # find start and end by multiplying the timestamps with the sampling rate for ema markers
        starting_point = math.floor(float(split_line[2]) * get_srate(int(split_line[0])))
        ending_point = math.ceil(float(split_line[3]) * get_srate(int(split_line[0])))
        
        # find start and end by multiplying the timestamps with the sampling rate for audio
        start_point = (math.floor(float(split_line[2]) * 22050)) - 220
        end_point = (math.ceil(float(split_line[3]) * 22050)) + 220
  
        # get data frame with all values for ema marker
        ul = get_values(UL_df, merged_ul, file_code)
        ll = get_values(LL_df, merged_ll, file_code)
        jw = get_values(JW_df, merged_jw, file_code)
        td = get_values(TD_df, merged_td, file_code)
        tb = get_values(TB_df, merged_tb, file_code)
        tt = get_values(TT_df, merged_tt, file_code)
        segment = get_values_audio(audio_df, merged_audio, file_code)

        # also add a key-value pair for mel spectogram
        raw_audio = segment[0].values
        mel_signal = logmelfilterbank(raw_audio, 22050)
        
        # concate all dataframes with ema markers
        df_data = pd.concat([ul, ll, jw, td, tb, tt], axis = 1)
        df_data = df_data.loc[:,~df_data.columns.duplicated()]
        
        # retrieve meta data and combine with ema data into dictionary
        data = {'word' : [split_line[1]],
                'srate': [get_srate(int(split_line[0]))],
                'sent' : [sent_number],
                'file' : [file_code],
                'Data'   : [df_data],
                'Audio' : [segment],
                'Mel' : [mel_signal] } 
        
        ema_frames.append(data)

0


C:\Users\nchau\anaconda3\envs\env_pytorch\lib\site-packages\ipykernel_launcher.py:129: FutureWarning: Pass sr=22050, n_fft=1024, n_mels=80, fmin=0, fmax=11025.0 as keyword args. From version 0.10 passing these as positional arguments will result in an error


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

In [12]:
%store ema_frames

Stored 'ema_frames' (list)


In [2]:
# FUNCTION TO ADD FILE CODE TO EMA DATA
def add_file(dataframe): 
    ''' 
        Using the list, get the index as the file code and add a new column with this index information
    '''
    for idx, frame in enumerate(dataframe):
        frame['File code'] = idx
        first_column = frame.pop('File code')
        frame.insert(0, 'File code', first_column)
        

# FUNCTION TO ADD FILE CODE TO AUDIO DATA   
def add_file_audio(dataframe): 
    ''' 
        Using the list, get the index as the file code and add a new column with this index information
    '''
    for idx, frame in enumerate(dataframe):
        data_frame = frame.insert(0, 'File Code', idx)
        
# FUNCTION FOR APPENDING LIST        
def append_list(position):
    '''
        Append the list with the mat data and return list with dataframes from each file
    '''
    if(position == 0):
        list_df = audio_df.append(pd.DataFrame.from_dict(data[0][0][2]))
    elif(position == 1):
        list_df = UL_df.append(pd.DataFrame.from_dict(data[0][1][2]))
    elif(position == 2):
        list_df = LL_df.append(pd.DataFrame.from_dict(data[0][2][2]))
    elif(position == 3):
        list_df = JW_df.append(pd.DataFrame.from_dict(data[0][3][2]))
    elif(position == 4):
        list_df = TD_df.append(pd.DataFrame.from_dict(data[0][4][2]))
    elif(position == 5):
        list_df = TB_df.append(pd.DataFrame.from_dict(data[0][5][2]))
    elif(position == 6):
        list_df = TT_df.append(pd.DataFrame.from_dict(data[0][6][2]))
    
    return list_df

# FUNCTION FOR PREPROCESSING DATA 
def preprocess(list_name, ema_name):
    '''
        Prepare the data to be used.
    '''
    # add file code to the dataframes and merge all files
    add_file(list_name)
    merged_file = pd.concat(list_name, axis = 0)
    
    # init column names and rename 
    column1 = "{}_0".format(ema_name)
    column2 = "{}_1".format(ema_name)
    column3 = "{}_2".format(ema_name)
    merged_file.columns = ['File Code', column1 , column2, column3]
    
    return merged_file

def preprocess_audio(list_name):
    '''
         Preprocessing steps of audio dataframes
    ''' 
    add_file_audio(list_name)
    merged_file_audio = pd.concat(list_name, axis = 0)
    
    return merged_file_audio
        
# FUNCTION FOR OBTAINING SAMPLING RATE
def get_srate(file_number):
    '''
        From the ema files get the sampling rate
    ''' 
    directory = 'data/Data/F1/mat'
    
    # still needs to ignore the .DS_Store file in a better way
    file = sorted(os.listdir(directory))[file_number + 1]
    
    f = os.path.join(directory, file)
    mat = scipy.io.loadmat(f)['usctimit_ema_f1_{:03}_{:03}'.format(file_number*5 + 1, file_number*5 + 5)]
    
    #returns the srate which is stored here
    return mat[0][1][1][0][0]

# FUNCTION FOR GETTING COORIDNATES FOR WORD TIMEFRAME
def get_values(df_list, merged_df, file_code): 
    '''
        Goes through list of dataframe and if it matches the file code it gets the coordinates for th
    '''
    for i in range(len(df_list)):
        if (file_code == i):
            current_df = merged_df.loc[merged_df['File Code'] == i]
            current_var = current_df.iloc[starting_point:ending_point, :]
            
    return current_var

# FUNCTION FOR GETTING COORIDNATES FOR WORD AUDIO SEGMENT
def get_values_audio(a_df, merged_a, file_code): 
    '''
        Goes through list of dataframe and if it matches the file code it gets the coordinates for th
    '''
    for i in range(len(a_df)):
        if (file_code == i):
            current_df = merged_a.loc[merged_a['File Code'] == i]
            segment = current_df.iloc[start_point:end_point, :]
    return segment

# convert raw audio to mel_spectogram
def logmelfilterbank(audio,
                     sampling_rate,
                     fft_size=1024,
                     hop_size=256,
                     win_length=None,
                     window="hann",
                     num_mels=80,
                     fmin=None,
                     fmax=None,
                     eps=1e-10,
                     ):
    ''' essentially the same as librosa.feature.melspectrogram + log10 '''

    # get amplitude spectrogram
    x_stft = librosa.stft(audio, n_fft=fft_size, hop_length=hop_size,
                          win_length=win_length, window=window, pad_mode="reflect")
    spc = np.abs(x_stft).T  # (#frames, #bins)

    # get mel basis
    fmin = 0 if fmin is None else fmin
    fmax = sampling_rate / 2 if fmax is None else fmax
    mel_basis = librosa.filters.mel(sampling_rate, fft_size, num_mels, fmin, fmax)

    return np.log10(np.maximum(eps, np.dot(spc, mel_basis.T)))

# Create X variable 
def get_features(ema_list):
    ''' 
        Obtain features and dereive longest length of an array
    '''
    features=[] #list to save features
    features_length=[] #list to save labels
    
    for dct in ema_list:
        features_length.append(len(dct['Mel'][0]))
        features.append(dct['Mel'][0]) # array
        
    return(np.array(features), features_length)